### Задача

Проанализировать базу данных сервиса для чтения книг по подписке, чтобы на основе этого анализа компания смогла сформулировать предложение для нового продукта.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

#### Предварительный осмотр данных

In [2]:
query = '''

SELECT *
FROM books
LIMIT 2'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336


In [3]:
query = '''

SELECT *
FROM authors
LIMIT 2

'''

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs


In [4]:
query = '''

SELECT *
FROM publishers
LIMIT 2

'''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book


In [5]:
query = '''

SELECT *
FROM ratings
LIMIT 2

'''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2


In [6]:
query = '''

SELECT *
FROM reviews
LIMIT 2

'''

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...


### Задания

#### Посчитаем, сколько книг вышло после 1 января 2000 года

In [7]:
query = '''

SELECT COUNT(DISTINCT(book_id))
FROM books
WHERE publication_date > '2000-01-01'
LIMIT 3

'''

pd.io.sql.read_sql(query, con = engine)

,count
0,819


После 1 января 2000 года вышло 819 книг с различным идентификатором

#### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [8]:
query = '''

SELECT b.book_id, 
       COUNT(DISTINCT rev.review_id) AS reviews_count, 
       AVG(rat.rating) AS rating_avg

FROM books AS b

LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
LEFT JOIN  ratings AS rat ON b.book_id = rat.book_id

GROUP BY b.book_id

'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,reviews_count,rating_avg
0,1,2,3.666667
1,2,1,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,4,4.000000
...,...,...,...
995,996,3,3.666667
996,997,3,3.400000
997,998,4,3.200000
998,999,2,4.500000


In [9]:
query = '''

with t1 as
        (SELECT b.book_id, 
               COUNT(DISTINCT rev.review_id) AS reviews_count, 
               AVG(rat.rating) AS rating_avg

        FROM books AS b

        LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
        LEFT JOIN  ratings AS rat ON b.book_id = rat.book_id

        GROUP BY b.book_id)
        
SELECT *
FROM t1
ORDER BY rating_avg

'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,reviews_count,rating_avg
0,303,2,1.50
1,371,2,2.00
2,316,2,2.00
3,202,3,2.00
4,915,3,2.25
...,...,...,...
995,421,2,5.00
996,418,2,5.00
997,169,2,5.00
998,901,2,5.00


In [10]:
query = '''

with t1 as
        (SELECT b.title,
                b.book_id,
               COUNT(DISTINCT rev.review_id) AS reviews_count, 
               AVG(rat.rating) AS rating_avg

        FROM books AS b

        LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
        LEFT JOIN  ratings AS rat ON b.book_id = rat.book_id

        GROUP BY b.book_id)
        
SELECT *
FROM t1
ORDER BY reviews_count

'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,title,book_id,reviews_count,rating_avg
0,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.666667
1,The Cat in the Hat and Other Dr. Seuss Favorites,672,0,5.000000
2,Essential Tales and Poems,221,0,4.000000
3,The Natural Way to Draw,808,0,3.000000
4,Disney's Beauty and the Beast (A Little Golden...,191,0,4.000000
...,...,...,...,...
995,The Giver (The Giver #1),733,6,3.750000
996,The Road,854,6,3.772727
997,The Curious Incident of the Dog in the Night-Time,695,6,4.081081
998,The Da Vinci Code (Robert Langdon #2),696,6,3.830508


*Средний рейтинг книг распределен в диапазоне 1.5-5, а количество обзоров - от 0 до 7*

#### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц, чтобы исключить из анализа брошюры

In [11]:
# сначала получим id издательства
query = '''

SELECT publisher_id,
COUNT(book_id) AS book_count
FROM books
WHERE num_pages > 50
GROUP BY publisher_id
ORDER BY book_count DESC   
LIMIT 1

'''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,book_count
0,212,42


In [12]:
# теперь выведем его название
query = '''

SELECT publisher

FROM publishers
WHERE publisher_id=212

'''

pd.io.sql.read_sql(query, con = engine)

,publisher
0,Penguin Books


*Penguin Books выпустило наибольшее количество книг (42)*

#### Определим автора с самой высокой средней оценкой книг, нужно учесть только книги с 50 и более оценками

In [13]:
# сначала получим id книг с 50 и более оценками
query = '''

SELECT book_id
FROM ratings
GROUP BY book_id
HAVING COUNT(rating_id) >= 50

'''

pd.io.sql.read_sql(query, con = engine)

,book_id
0,75
1,750
2,545
3,948
4,488
5,696
6,722
7,627
8,733
9,779


In [14]:
# теперь объединим таблицы ratings и books, оставив только нужные книги и присоединим таблицу с авторами

query = '''

WITH t1 AS (
        SELECT *
        FROM ratings AS rat
        INNER JOIN books AS b ON b.book_id=rat.book_id
        WHERE b.book_id IN (SELECT book_id
                            FROM ratings
                            GROUP BY book_id
                            HAVING COUNT(rating_id) >= 50
                            ))
        
SELECT author,
    AVG(rating) AS avg_rating
FROM authors AS a
INNER JOIN t1 ON a.author_id=t1.author_id
GROUP BY author
ORDER BY avg_rating DESC
LIMIT 1
'''

pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


J.K. Rowling и Mary GrandPré авторы с самой высокой средней оценкой книг

####  Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [15]:
# сначала получим username пользователей, которые поставили больше 48 оценок¶
query = '''

SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 48

'''

pd.io.sql.read_sql(query, con = engine)

,username
0,walter49
1,sfitzgerald
2,jennifermiller
3,andreaeaton
4,xdavis
5,vcarter
6,paul88
7,lesliegibbs
8,susan85
9,martinadam


In [16]:
# найдем количество обзоров для каждого из этих пользователей
query = '''

SELECT username,
        COUNT(review_id)
FROM reviews

WHERE username IN (SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating_id) > 48)
GROUP BY username

'''

pd.io.sql.read_sql(query, con = engine)


,username,count
0,walter49,23
1,sfitzgerald,28
2,jennifermiller,25
3,andreaeaton,22
4,xdavis,18
5,vcarter,20
6,paul88,22
7,lesliegibbs,26
8,susan85,29
9,martinadam,27


In [17]:
# найдем среднее количество обзоров
query = '''
WITH t1 as
        (SELECT username,
                COUNT(review_id)
        FROM reviews

        WHERE username IN (SELECT username
                            FROM ratings
                            GROUP BY username
                            HAVING COUNT(rating_id) > 48)
        GROUP BY username)
SELECT AVG(count)
FROM t1

'''

pd.io.sql.read_sql(query, con = engine)

,avg
0,24.0


*Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок – 24*

In [18]:

query = '''
SELECT *
FROM books
'''

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


## Выводы

База данных содержит 1000 книг, 819 из которых вышли после 1 января 2000 года

Средний рейтинг книг распределен в диапазоне 1.5-5, а количество обзоров на них от 0 до 7

Наибольшее количество книг (42) принадлежит издательству Penguin Books

J.K. Rowling и Mary GrandPré авторы с самой высокой средней оценкой книг 4.29

Cреднее количество обзоров от самых активных пользователей (которые поставили больше 48 оценок) – 24